# Demostración de Threading con Estado Compartido

Este notebook demuestra cómo ejecutar dos hilos simultáneamente y manejar adecuadamente el estado de una variable compartida usando sincronización.

In [3]:
import threading
import time
import random
from datetime import datetime

# Variables compartidas y locks para sincronización
source_array = list(range(1, 11))  # Array del 1 al 10
destination_array = []  # Variable compartida donde se añaden los elementos
array_lock = threading.Lock()  # Lock para proteger ambos arrays
execution_log = []
log_lock = threading.Lock()

def thread_function_1(name, max_operations=5):
    """Función del primer hilo que saca elementos del array fuente"""
    global source_array, destination_array
    
    operations_count = 0
    while operations_count < max_operations:
        # Simular trabajo antes de intentar acceder
        time.sleep(random.uniform(0.1, 0.3))
        
        element_extracted = None
        source_state = None
        dest_state = None
        
        # Acceso seguro y ATÓMICO a las variables compartidas
        with array_lock:
            if source_array:  # Si hay elementos disponibles
                element_extracted = source_array.pop(0)  # Sacar el primer elemento
                destination_array.append(element_extracted)  # Añadirlo al array destino
                source_state = source_array.copy()  # Capturar estado actual
                dest_state = destination_array.copy()  # Capturar estado actual
                operations_count += 1
            else:
                # No hay más elementos disponibles
                source_state = []
                dest_state = destination_array.copy()
                break
        
        # Log y print FUERA del lock crítico para evitar problemas
        if element_extracted is not None:
            timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
            
            with log_lock:
                execution_log.append({
                    'thread': name,
                    'operation': f'EXTRAJO {element_extracted}',
                    'source_remaining': len(source_state),
                    'destination_count': len(dest_state),
                    'destination_content': dest_state,
                    'timestamp': timestamp
                })
            
            print(f"[{timestamp}] {name} - Extrajo elemento {element_extracted}")
            print(f"                          Array fuente: {source_state}")
            print(f"                          Array destino: {dest_state}")
        else:
            timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
            with log_lock:
                execution_log.append({
                    'thread': name,
                    'operation': 'SIN ELEMENTOS',
                    'source_remaining': 0,
                    'destination_count': len(dest_state),
                    'destination_content': dest_state,
                    'timestamp': timestamp
                })
            print(f"[{timestamp}] {name} - No hay más elementos disponibles")

def thread_function_2(name, max_operations=5):
    """Función del segundo hilo que saca elementos del array fuente"""
    global source_array, destination_array
    
    operations_count = 0
    while operations_count < max_operations:
        # Simular trabajo antes de intentar acceder
        time.sleep(random.uniform(0.1, 0.3))
        
        element_extracted = None
        source_state = None
        dest_state = None
        
        # Acceso seguro y ATÓMICO a las variables compartidas
        with array_lock:
            if source_array:  # Si hay elementos disponibles
                element_extracted = source_array.pop(0)  # Sacar el primer elemento
                destination_array.append(element_extracted)  # Añadirlo al array destino
                source_state = source_array.copy()  # Capturar estado actual
                dest_state = destination_array.copy()  # Capturar estado actual
                operations_count += 1
            else:
                # No hay más elementos disponibles
                source_state = []
                dest_state = destination_array.copy()
                break
        
        # Log y print FUERA del lock crítico para evitar problemas
        if element_extracted is not None:
            timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
            
            with log_lock:
                execution_log.append({
                    'thread': name,
                    'operation': f'EXTRAJO {element_extracted}',
                    'source_remaining': len(source_state),
                    'destination_count': len(dest_state),
                    'destination_content': dest_state,
                    'timestamp': timestamp
                })
            
            print(f"[{timestamp}] {name} - Extrajo elemento {element_extracted}")
            print(f"                          Array fuente: {source_state}")
            print(f"                          Array destino: {dest_state}")
        else:
            timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
            with log_lock:
                execution_log.append({
                    'thread': name,
                    'operation': 'SIN ELEMENTOS',
                    'source_remaining': 0,
                    'destination_count': len(dest_state),
                    'destination_content': dest_state,
                    'timestamp': timestamp
                })
            print(f"[{timestamp}] {name} - No hay más elementos disponibles")

# Resetear variables para nueva ejecución
source_array = list(range(1, 11))  # Reiniciar array del 1 al 10
destination_array = []
execution_log = []

print("=== DEMOSTRACIÓN DE HILOS CON ARRAYS COMPARTIDOS (CORREGIDA) ===")
print(f"Array fuente inicial: {source_array}")
print(f"Array destino inicial: {destination_array}")
print("\n--- Creando y ejecutando hilos ---")

# Crear los hilos
thread1 = threading.Thread(target=thread_function_1, args=("HILO-1", 6))
thread2 = threading.Thread(target=thread_function_2, args=("HILO-2", 6))

# Iniciar ambos hilos simultáneamente
start_time = time.time()
thread1.start()
thread2.start()

# Esperar a que ambos hilos terminen
thread1.join()
thread2.join()
end_time = time.time()

print("\n=== RESULTADOS FINALES ===")
print(f"Array fuente final: {source_array}")
print(f"Array destino final: {destination_array}")
print(f"Elementos procesados: {len(destination_array)}")
print(f"Elementos restantes en fuente: {len(source_array)}")
print(f"Tiempo total de ejecución: {end_time - start_time:.2f} segundos")

print("\n--- Log cronológico de operaciones ---")
# Ordenar por timestamp para mostrar secuencia real
execution_log.sort(key=lambda x: x['timestamp'])
for entry in execution_log:
    print(f"[{entry['timestamp']}] {entry['thread']} - {entry['operation']}")

print("\n=== VERIFICACIÓN DE INTEGRIDAD ===")
# Verificar que todos los elementos fueron procesados correctamente
expected_total = 10
actual_total = len(source_array) + len(destination_array)
print(f"Total esperado de elementos: {expected_total}")
print(f"Total actual de elementos: {actual_total}")
print(f"Integridad mantenida: {expected_total == actual_total}")

# Verificar que no hay duplicados en el array destino
unique_elements = set(destination_array)
print(f"Elementos únicos en destino: {len(unique_elements)}")
print(f"Total elementos en destino: {len(destination_array)}")
print(f"Sin duplicados: {len(unique_elements) == len(destination_array)}")

# Verificar orden correcto (debería ser 1,2,3,4,5,6,7,8,9,10)
is_sequential = destination_array == sorted(destination_array)
print(f"Elementos extraídos en orden secuencial: {is_sequential}")

# Estadísticas por hilo
thread1_ops = [entry for entry in execution_log if entry['thread'] == 'HILO-1' and 'EXTRAJO' in entry['operation']]
thread2_ops = [entry for entry in execution_log if entry['thread'] == 'HILO-2' and 'EXTRAJO' in entry['operation']]

print(f"\nOperaciones exitosas HILO-1: {len(thread1_ops)}")
print(f"Operaciones exitosas HILO-2: {len(thread2_ops)}")
print(f"Total operaciones exitosas: {len(thread1_ops) + len(thread2_ops)}")

print("\n¡Demostración completada con sincronización corregida!")
print("Los hilos ahora procesan elementos de forma completamente segura y atómica.")

=== DEMOSTRACIÓN DE HILOS CON ARRAYS COMPARTIDOS (CORREGIDA) ===
Array fuente inicial: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Array destino inicial: []

--- Creando y ejecutando hilos ---
[16:57:40.974] HILO-2 - Extrajo elemento 1
                          Array fuente: [2, 3, 4, 5, 6, 7, 8, 9, 10]
                          Array destino: [1]
[16:57:40.994] HILO-1 - Extrajo elemento 2
                          Array fuente: [3, 4, 5, 6, 7, 8, 9, 10]
                          Array destino: [1, 2]
[16:57:41.153] HILO-1 - Extrajo elemento 3
                          Array fuente: [4, 5, 6, 7, 8, 9, 10]
                          Array destino: [1, 2, 3]
[16:57:40.974] HILO-2 - Extrajo elemento 1
                          Array fuente: [2, 3, 4, 5, 6, 7, 8, 9, 10]
                          Array destino: [1]
[16:57:40.994] HILO-1 - Extrajo elemento 2
                          Array fuente: [3, 4, 5, 6, 7, 8, 9, 10]
                          Array destino: [1, 2]
[16:57:41.153] HILO-1 - Extrajo el

In [4]:
# ===== LIMPIEZA COMPLETA DE ESTADO =====
import threading
import time
import gc

print("🧹 LIMPIEZA DE ESTADO PREVIA A EJECUCIÓN")

# Verificar hilos activos antes de empezar
active_threads_before = threading.active_count()
print(f"Hilos activos antes de limpiar: {active_threads_before}")

# Limpiar completamente todas las variables globales
try:
    del source_array, destination_array, execution_log, array_lock, log_lock
    print("✅ Variables globales eliminadas")
except NameError:
    print("ℹ️  Variables globales no existían previamente")

# Forzar recolección de basura
gc.collect()

# Esperar un momento para que cualquier hilo termine
time.sleep(0.5)

# Verificar hilos activos después de la limpieza
active_threads_after = threading.active_count()
print(f"Hilos activos después de limpiar: {active_threads_after}")

# Reinicializar completamente todas las variables
source_array = list(range(1, 11))
destination_array = []
execution_log = []
array_lock = threading.Lock()
log_lock = threading.Lock()

print("🔄 Estado completamente reinicializado")
print(f"Array fuente: {source_array}")
print(f"Array destino: {destination_array}")
print(f"Log de ejecución: {len(execution_log)} entradas")
print("=" * 50)

🧹 LIMPIEZA DE ESTADO PREVIA A EJECUCIÓN
Hilos activos antes de limpiar: 6
✅ Variables globales eliminadas
Hilos activos después de limpiar: 6
🔄 Estado completamente reinicializado
Array fuente: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Array destino: []
Log de ejecución: 0 entradas


In [5]:
import threading
import time
import random
from datetime import datetime
import uuid

# Generar ID único para esta ejecución
execution_id = str(uuid.uuid4())[:8]
print(f"🆔 ID de ejecución: {execution_id}")

def thread_function_enhanced(name, max_operations=5):
    """Función mejorada con verificaciones adicionales de seguridad"""
    global source_array, destination_array
    
    operations_count = 0
    thread_id = f"{name}-{execution_id}"
    
    print(f"🚀 {thread_id} iniciado")
    
    while operations_count < max_operations:
        # Simular trabajo antes de intentar acceder
        time.sleep(random.uniform(0.1, 0.3))
        
        element_extracted = None
        source_state = None
        dest_state = None
        
        # Verificar que aún estamos en la misma ejecución
        current_execution = f"{name}-{execution_id}"
        
        # Acceso seguro y ATÓMICO a las variables compartidas
        with array_lock:
            if source_array:  # Si hay elementos disponibles
                element_extracted = source_array.pop(0)
                destination_array.append(element_extracted)
                source_state = source_array.copy()
                dest_state = destination_array.copy()
                operations_count += 1
            else:
                source_state = []
                dest_state = destination_array.copy()
                break
        
        # Log y print FUERA del lock crítico
        if element_extracted is not None:
            timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
            
            with log_lock:
                execution_log.append({
                    'execution_id': execution_id,
                    'thread': thread_id,
                    'operation': f'EXTRAJO {element_extracted}',
                    'source_remaining': len(source_state),
                    'destination_count': len(dest_state),
                    'timestamp': timestamp
                })
            
            print(f"[{timestamp}] {thread_id} - Extrajo elemento {element_extracted}")
            print(f"                          Array fuente: {source_state}")
            print(f"                          Array destino: {dest_state}")
        else:
            timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
            with log_lock:
                execution_log.append({
                    'execution_id': execution_id,
                    'thread': thread_id,
                    'operation': 'SIN ELEMENTOS',
                    'source_remaining': 0,
                    'destination_count': len(dest_state),
                    'timestamp': timestamp
                })
            print(f"[{timestamp}] {thread_id} - No hay más elementos disponibles")
    
    print(f"🏁 {thread_id} terminado - {operations_count} operaciones realizadas")

print(f"\n=== DEMOSTRACIÓN DE HILOS (ID: {execution_id}) ===")
print(f"Array fuente inicial: {source_array}")
print(f"Array destino inicial: {destination_array}")
print(f"Hilos activos al inicio: {threading.active_count()}")
print("\n--- Creando y ejecutando hilos ---")

# Crear los hilos con nombres únicos
thread1 = threading.Thread(target=thread_function_enhanced, args=("HILO-1", 6))
thread2 = threading.Thread(target=thread_function_enhanced, args=("HILO-2", 6))

# Marcar los hilos como daemon para que terminen cuando termine el programa principal
thread1.daemon = True
thread2.daemon = True

# Iniciar ambos hilos simultáneamente
start_time = time.time()
thread1.start()
thread2.start()

print(f"✅ Hilos iniciados. Hilos activos: {threading.active_count()}")

# Esperar a que ambos hilos terminen con timeout de seguridad
thread1.join(timeout=10)
thread2.join(timeout=10)

end_time = time.time()

print(f"\n💯 Hilos finalizados. Hilos activos: {threading.active_count()}")

print(f"\n=== RESULTADOS FINALES (ID: {execution_id}) ===")
print(f"Array fuente final: {source_array}")
print(f"Array destino final: {destination_array}")
print(f"Elementos procesados: {len(destination_array)}")
print(f"Elementos restantes en fuente: {len(source_array)}")
print(f"Tiempo total de ejecución: {end_time - start_time:.2f} segundos")

# Filtrar logs solo de esta ejecución
current_execution_logs = [entry for entry in execution_log if entry.get('execution_id') == execution_id]

print(f"\n--- Log cronológico (ID: {execution_id}) ---")
current_execution_logs.sort(key=lambda x: x['timestamp'])
for entry in current_execution_logs:
    print(f"[{entry['timestamp']}] {entry['thread']} - {entry['operation']}")

print(f"\n=== VERIFICACIÓN DE INTEGRIDAD (ID: {execution_id}) ===")
expected_total = 10
actual_total = len(source_array) + len(destination_array)
print(f"Total esperado de elementos: {expected_total}")
print(f"Total actual de elementos: {actual_total}")
print(f"Integridad mantenida: {expected_total == actual_total}")

# Verificaciones adicionales
unique_elements = set(destination_array)
print(f"Elementos únicos en destino: {len(unique_elements)}")
print(f"Total elementos en destino: {len(destination_array)}")
print(f"Sin duplicados: {len(unique_elements) == len(destination_array)}")

is_sequential = destination_array == sorted(destination_array)
print(f"Elementos extraídos en orden secuencial: {is_sequential}")

# Estadísticas por hilo (solo de esta ejecución)
thread1_ops = [entry for entry in current_execution_logs if 'HILO-1' in entry['thread'] and 'EXTRAJO' in entry['operation']]
thread2_ops = [entry for entry in current_execution_logs if 'HILO-2' in entry['thread'] and 'EXTRAJO' in entry['operation']]

print(f"\nOperaciones exitosas HILO-1: {len(thread1_ops)}")
print(f"Operaciones exitosas HILO-2: {len(thread2_ops)}")
print(f"Total operaciones exitosas: {len(thread1_ops) + len(thread2_ops)}")
print(f"Entradas en log de esta ejecución: {len(current_execution_logs)}")

print(f"\n🎉 ¡Demostración completada! (ID: {execution_id})")
print("Con verificaciones mejoradas para evitar ejecuciones duplicadas.")

🆔 ID de ejecución: 06818f2f

=== DEMOSTRACIÓN DE HILOS (ID: 06818f2f) ===
Array fuente inicial: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Array destino inicial: []
Hilos activos al inicio: 6

--- Creando y ejecutando hilos ---
🚀 HILO-1-06818f2f iniciado
🚀 HILO-2-06818f2f iniciado
✅ Hilos iniciados. Hilos activos: 8
[17:00:35.006] HILO-1-06818f2f - Extrajo elemento 1
                          Array fuente: [2, 3, 4, 5, 6, 7, 8, 9, 10]
                          Array destino: [1]
[17:00:35.048] HILO-2-06818f2f - Extrajo elemento 2
                          Array fuente: [3, 4, 5, 6, 7, 8, 9, 10]
                          Array destino: [1, 2]
[17:00:35.179] HILO-1-06818f2f - Extrajo elemento 3
                          Array fuente: [4, 5, 6, 7, 8, 9, 10]
                          Array destino: [1, 2, 3]
[17:00:35.322] HILO-2-06818f2f - Extrajo elemento 4
                          Array fuente: [5, 6, 7, 8, 9, 10]
                          Array destino: [1, 2, 3, 4]
[17:00:35.394] HILO-1-06818f2